<a href="https://colab.research.google.com/github/lizbiz0510/Cornejo_A2/blob/main/Cornejo_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install any missing dependencies
! pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Read in all required CSV files

In [ ]:

import pandas as pd
import pandasql as ps



data1 = pd.read_csv('collaborative_book_metadata.csv')
print(data1.head().to_string()) 


   Unnamed: 0   book_id                                                              title                                                    image_url                                                                          url  num_pages  ratings_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [ ]:
data2 = pd.read_csv('collaborative_books_df.csv')
print(data2.head().to_string()) 

   Unnamed: 0               title  book_id  user_id_mapping  book_id_mapping  Predicted Rating  Actual Rating
0           0  I Am the Messenger    19057             1537              299               4.5              5
1           1  I Am the Messenger    19057            23039              299               4.9              3
2           2  I Am the Messenger    19057            39096              299               3.9              3
3           3  I Am the Messenger    19057            14631              299               4.7              4
4           4  I Am the Messenger    19057            32816              299               4.3              5


Query for the data used in training/testing

In [10]:
#input1
sql_query = "SELECT d1.book_id,`Predicted Rating`, ratings_count,num_pages, CASE WHEN `Actual Rating` < 3 THEN 'False' ELSE 'True' END AS Recommended FROM data1 d1 JOIN data2 d2 ON d1.book_id = d2.book_id LIMIT 5000"
df = ps.sqldf(sql_query,locals())
print(df)

      book_id  Predicted Rating  ratings_count  num_pages Recommended
0     5899779               1.3         105537        320       False
1     5899779               1.6         105537        320        True
2     5899779               2.0         105537        320       False
3     5899779               2.0         105537        320        True
4     5899779               2.1         105537        320       False
...       ...               ...            ...        ...         ...
4995  4502877               4.4         138855        264        True
4996  4502877               4.4         138855        264        True
4997  4502877               4.4         138855        264        True
4998  4502877               4.4         138855        264        True
4999  4502877               4.5         138855        264        True

[5000 rows x 5 columns]


In [11]:
#input2
sql_query = "SELECT d1.book_id,`Predicted Rating`, ratings_count,num_pages, CASE WHEN `Actual Rating` < 2 THEN 'False' ELSE 'True' END AS Recommended FROM data1 d1 JOIN data2 d2 ON d1.book_id = d2.book_id LIMIT 5000"
df = ps.sqldf(sql_query,locals())
print(df)

      book_id  Predicted Rating  ratings_count  num_pages Recommended
0     5899779               1.3         105537        320       False
1     5899779               1.6         105537        320        True
2     5899779               2.0         105537        320        True
3     5899779               2.0         105537        320        True
4     5899779               2.1         105537        320       False
...       ...               ...            ...        ...         ...
4995  4502877               4.4         138855        264        True
4996  4502877               4.4         138855        264        True
4997  4502877               4.4         138855        264        True
4998  4502877               4.4         138855        264        True
4999  4502877               4.5         138855        264        True

[5000 rows x 5 columns]


Split dataset in to randomized train and test sets

In [13]:
#input3
sql_query = "SELECT d1.book_id,`Predicted Rating`, ratings_count,num_pages, CASE WHEN `Actual Rating` < 5 THEN 'False' ELSE 'True' END AS Recommended FROM data1 d1 JOIN data2 d2 ON d1.book_id = d2.book_id LIMIT 5000"
df = ps.sqldf(sql_query,locals())
print(df)

      book_id  Predicted Rating  ratings_count  num_pages Recommended
0     5899779               1.3         105537        320       False
1     5899779               1.6         105537        320        True
2     5899779               2.0         105537        320       False
3     5899779               2.0         105537        320       False
4     5899779               2.1         105537        320       False
...       ...               ...            ...        ...         ...
4995  4502877               4.4         138855        264        True
4996  4502877               4.4         138855        264        True
4997  4502877               4.4         138855        264        True
4998  4502877               4.4         138855        264        True
4999  4502877               4.5         138855        264       False

[5000 rows x 5 columns]


In [14]:
trainset = df.sample(frac=0.1)
df = df.drop(trainset.index)
testset = df.sample(frac=0.75)
sampleset = df.drop(testset.index)
print('Length of Trainset:', len(trainset),'\nLength of Testset:', len(testset),'\nLength of Sample Set:', len(sampleset))

Length of Trainset: 500 
Length of Testset: 3375 
Length of Sample Set: 1125


Classify points in test set using data in trainset and calculate accuracy

In [15]:


# Running KNN Algorithm
def runKNN(trainset,testset):
  recommededbooks = []
  #Hyperparameters 
  K = 3
  correct = 0
  wrong = 0
  i = 1
  for index, row in testset.iterrows():
    distances = []
    for index2, row2 in trainset.iterrows():
      dist = (row['Predicted Rating'] - row2['Predicted Rating'])**2 + (row['ratings_count'] - row2['ratings_count'])**2 + (row['num_pages'] - row2['num_pages'])**2
      distances.append({'dist': dist, 'label': row2['Recommended']})
    distances = sorted(distances, key = lambda x: x['dist'])
    distances = distances[:K]

    n = 0
    for item in distances:
      if item['label'] == 'True':
        n += 1

    if n >= K//2 + 1:
      if row['book_id'] not in recommededbooks:
        recommededbooks.append(row['book_id'])
      if row['Recommended'] == 'True':
        correct += 1
      else:
        wrong += 1
      print('Datapoint: ', i, ',This test item is Reccommended', row['Recommended'])
    else:
      if row['Recommended'] == 'False':
        correct += 1
      else:
        wrong += 1
      print('Datapoint: ', i, ',This test item is Not Recommended', row['Recommended'])
    i += 1
    
  accuracy = correct / (correct + wrong)
  print('Accuracy:' , accuracy * 100)
  return recommededbooks

    

In [16]:
runKNN(trainset,testset)

Datapoint:  1 ,This test item is Not Recommended False
Datapoint:  2 ,This test item is Not Recommended True
Datapoint:  3 ,This test item is Not Recommended False
Datapoint:  4 ,This test item is Not Recommended True
Datapoint:  5 ,This test item is Reccommended False
Datapoint:  6 ,This test item is Not Recommended False
Datapoint:  7 ,This test item is Not Recommended False
Datapoint:  8 ,This test item is Not Recommended False
Datapoint:  9 ,This test item is Not Recommended False
Datapoint:  10 ,This test item is Not Recommended False
Datapoint:  11 ,This test item is Not Recommended False
Datapoint:  12 ,This test item is Not Recommended False
Datapoint:  13 ,This test item is Not Recommended False
Datapoint:  14 ,This test item is Reccommended True
Datapoint:  15 ,This test item is Reccommended True
Datapoint:  16 ,This test item is Not Recommended False
Datapoint:  17 ,This test item is Not Recommended False
Datapoint:  18 ,This test item is Not Recommended False
Datapoint:  19

[9520360,
 186074,
 29056083,
 6936382,
 233649,
 862041,
 16130549,
 13,
 66559,
 872333,
 15507958,
 10664113,
 2233407,
 7937462,
 41804,
 170210,
 7235533,
 10572,
 228665,
 110494,
 4502877,
 11564]

In [17]:
import numpy as np
#function defining the RS AI
def RSBookRecommeder(trainset, books):
  recommededbooks = runKNN(trainset,books)
  top_picks = np.random.choice(recommededbooks, size = 3)
  return top_picks




In [18]:
#3 exeuctions of AI inputs and outputs
top_picks = RSBookRecommeder(trainset, sampleset.sample(n= 50))
print('The RS AI recommended these 3 book ids.', top_picks, '\n\n')
top_picks = RSBookRecommeder(trainset, sampleset.sample(n= 50))
print('The RS AI recommended these 3 book ids.', top_picks, '\n\n')
top_picks = RSBookRecommeder(trainset, sampleset.sample(n= 50))
print('The RS AI recommended these 3 book ids.', top_picks, '\n\n')


Datapoint:  1 ,This test item is Not Recommended False
Datapoint:  2 ,This test item is Not Recommended False
Datapoint:  3 ,This test item is Not Recommended False
Datapoint:  4 ,This test item is Reccommended False
Datapoint:  5 ,This test item is Reccommended True
Datapoint:  6 ,This test item is Reccommended True
Datapoint:  7 ,This test item is Not Recommended True
Datapoint:  8 ,This test item is Not Recommended True
Datapoint:  9 ,This test item is Not Recommended False
Datapoint:  10 ,This test item is Not Recommended False
Datapoint:  11 ,This test item is Not Recommended True
Datapoint:  12 ,This test item is Not Recommended False
Datapoint:  13 ,This test item is Reccommended True
Datapoint:  14 ,This test item is Reccommended False
Datapoint:  15 ,This test item is Reccommended True
Datapoint:  16 ,This test item is Reccommended True
Datapoint:  17 ,This test item is Reccommended False
Datapoint:  18 ,This test item is Not Recommended False
Datapoint:  19 ,This test item is